In [2]:
import matplotlib.pyplot as plt
import pandas as pd
from processing import biosppy_processing, nk_processing, processing_aggregate
from time_domain import time
from frequency_domain import frequency
from nonlinear import nonlinear
import numpy as np

from tqdm.notebook import tqdm

pd.set_option('display.float_format', lambda x: f'{x:.2f}') 

## 0. Read Data

In [3]:
sampling_rate = 2000
raw = pd.read_csv('./data/ecg_70.txt', sep = ' ').T
raw

,1,2,3,4,5,6,7,8,9,10,...,239991,239992,239993,239994,239995,239996,239997,239998,239999,240000
id1,0.79,0.79,0.79,0.78,0.78,0.77,0.77,0.76,0.76,0.75,...,1.75,1.69,1.62,1.56,1.49,1.43,1.36,1.30,1.23,1.16
id2,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,...,-0.02,-0.02,-0.02,-0.02,-0.02,-0.02,-0.02,-0.02,-0.02,-0.02
id3,0.04,0.04,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,...,0.13,0.13,0.13,0.13,0.13,0.13,0.13,0.13,0.13,0.13
id4,-0.36,-0.36,-0.35,-0.34,-0.34,-0.33,-0.33,-0.32,-0.32,-0.31,...,-0.20,-0.19,-0.19,-0.19,-0.19,-0.19,-0.19,-0.19,-0.19,-0.19
id5,-0.02,-0.02,-0.02,-0.02,-0.02,-0.01,-0.01,-0.01,-0.01,-0.01,...,0.10,0.08,0.06,0.03,0.01,-0.01,-0.03,-0.05,-0.07,-0.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
id69,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.09,0.09,0.10,...,0.08,0.08,0.08,0.08,0.08,0.08,0.08,0.08,0.08,0.08
id70,0.00,0.00,0.00,0.00,0.01,0.01,0.01,0.01,0.01,0.01,...,-0.07,-0.09,-0.10,-0.12,-0.13,-0.14,-0.15,-0.16,-0.16,-0.17
id71,-0.55,-0.55,-0.55,-0.55,-0.55,-0.55,-0.55,-0.56,-0.56,-0.56,...,0.08,0.08,0.08,0.08,0.08,0.08,0.08,0.08,0.08,0.08
id72,-0.00,-0.00,-0.00,-0.01,-0.01,-0.01,-0.01,-0.01,-0.01,-0.02,...,-0.07,-0.07,-0.07,-0.07,-0.07,-0.07,-0.07,-0.07,-0.08,-0.08


## 1. Time Domain

In [17]:
start = 0
start *= 2000
end = 10
end *= 2000
print(f'{start} ~ {end}')



time_nk_result = dict()
for row in tqdm(raw.iterrows(), total = len(raw)):
    id = row[0]
    sampling_rate = 2000
    signal = row[1].tolist()[start : end]
    pro = nk_processing(signal, sampling_rate)
    time_nk_result[id] = time(pro['rpeaks'], 2000)
    
    
time_biosppy_result = dict()
for row in tqdm(raw.iterrows(), total = len(raw)):
    id = row[0]
    sampling_rate = 2000
    signal = row[1].tolist()[start : end]
    pro = biosppy_processing(signal, sampling_rate)
    time_biosppy_result[id] = time(pro['rpeaks'], 2000)
    
    
time_aggregate_result = dict()
for row in tqdm(raw.iterrows(), total = len(raw)):
    id = row[0]
    sampling_rate = 2000
    signal = row[1].tolist()[start : end]
    pro = processing_aggregate(signal, sampling_rate)
    time_aggregate_result[id] = time(pro['rpeaks'], 2000)


0 ~ 20000


  0%|          | 0/70 [00:00<?, ?it/s]

  0%|          | 0/70 [00:00<?, ?it/s]

  0%|          | 0/70 [00:00<?, ?it/s]

In [18]:
print(f'{start} ~ {end}')
temp = time_nk_result
temp = pd.DataFrame(temp).T.describe().loc[['mean', 'std']].T
temp[temp.index.str.endswith('rmssd')]



0 ~ 20000


,mean,std
pyhrv rmssd,49.37,44.04
nk rmssd,49.37,44.04


In [19]:
print(f'{start} ~ {end}')
temp = time_biosppy_result
temp = pd.DataFrame(temp).T.describe().loc[['mean', 'std']].T
temp[temp.index.str.endswith('rmssd')]



0 ~ 20000


,mean,std
pyhrv rmssd,46.78,43.75
nk rmssd,46.78,43.75


In [20]:
print(f'{start} ~ {end}')
temp = time_aggregate_result
temp = pd.DataFrame(temp).T.describe().loc[['mean', 'std']].T
temp[temp.index.str.endswith('rmssd')]



0 ~ 20000


,mean,std
pyhrv rmssd,49.33,44.05
nk rmssd,49.33,44.05


## 2. Frequency Domain

In [9]:
start = 0
start *= 2000
end = 60
end *= 2000
print(f'{start} ~ {end}')



frequency_nk_result = dict()
for row in tqdm(raw.iterrows(), total = len(raw)):
    id = row[0]
    sampling_rate = 2000
    signal = row[1].tolist()[start : end]
    pro = nk_processing(signal, sampling_rate)
    frequency_nk_result_ = dict()
    output = frequency(pro['rpeaks'], 2000)
    frequency_nk_result_['pyhrv welch'] = output['pyhrv welch']['fft_ratio']
    frequency_nk_result_['pyhrv ar'] = output['pyhrv ar']['ar_ratio']
    frequency_nk_result_['nk welch'] = output['nk welch']['HRV_LFHF']
    frequency_nk_result_['nk fft'] = output['nk fft']['HRV_LFHF']
    frequency_nk_result[id] = frequency_nk_result_



frequency_biosppy_result = dict()
for row in tqdm(raw.iterrows(), total = len(raw)):
    id = row[0]
    sampling_rate = 2000
    signal = row[1].tolist()[start : end]
    pro = biosppy_processing(signal, sampling_rate)
    frequency_biosppy_result_ = dict()
    output = frequency(pro['rpeaks'], 2000)
    frequency_biosppy_result_['pyhrv welch'] = output['pyhrv welch']['fft_ratio']
    frequency_biosppy_result_['pyhrv ar'] = output['pyhrv ar']['ar_ratio']
    frequency_biosppy_result_['nk welch'] = output['nk welch']['HRV_LFHF']
    frequency_biosppy_result_['nk fft'] = output['nk fft']['HRV_LFHF']
    frequency_biosppy_result[id] = frequency_biosppy_result_




frequency_aggregate_result = dict()
for row in tqdm(raw.iterrows(), total = len(raw)):
    id = row[0]
    sampling_rate = 2000
    signal = row[1].tolist()[start : end]
    pro = processing_aggregate(signal, sampling_rate)
    frequency_aggregate_result_ = dict()
    output = frequency(pro['rpeaks'], 2000)
    frequency_aggregate_result_['pyhrv welch'] = output['pyhrv welch']['fft_ratio']
    frequency_aggregate_result_['pyhrv ar'] = output['pyhrv ar']['ar_ratio']
    frequency_aggregate_result_['nk welch'] = output['nk welch']['HRV_LFHF']
    frequency_aggregate_result_['nk fft'] = output['nk fft']['HRV_LFHF']
    frequency_aggregate_result[id] = frequency_aggregate_result_





0 ~ 120000


  0%|          | 0/70 [00:00<?, ?it/s]

  0%|          | 0/70 [00:00<?, ?it/s]

  0%|          | 0/70 [00:00<?, ?it/s]

In [10]:
print(f'{start} ~ {end}')
temp = frequency_nk_result
temp = pd.DataFrame(temp).T.describe().loc[['mean', 'std']].T
temp



0 ~ 120000


,mean,std
pyhrv welch,2.34,2.28
pyhrv ar,1.86,0.05
nk welch,1.68,1.83
nk fft,1.73,1.66


In [11]:
print(f'{start} ~ {end}')
temp = frequency_biosppy_result
temp = pd.DataFrame(temp).T.describe().loc[['mean', 'std']].T
temp



0 ~ 120000


,mean,std
pyhrv welch,2.33,2.33
pyhrv ar,1.87,0.05
nk welch,1.59,1.78
nk fft,1.68,1.63


In [12]:
print(f'{start} ~ {end}')
temp = frequency_aggregate_result
temp = pd.DataFrame(temp).T.describe().loc[['mean', 'std']].T
temp



0 ~ 120000


,mean,std
pyhrv welch,2.35,2.29
pyhrv ar,1.86,0.05
nk welch,1.65,1.72
nk fft,1.69,1.60


## 3. Nonlienar

In [13]:
start = 0
start *= 2000
end = 60
end *= 2000
print(f'{start} ~ {end}')



nonlinear_nk_result = dict()
for row in tqdm(raw.iterrows(), total = len(raw)):
    id = row[0]
    sampling_rate = 2000
    signal = row[1].tolist()[start : end]
    pro = nk_processing(signal, sampling_rate)
    nonlinear_nk_result_ = dict()
    output = nonlinear(pro['rpeaks'], 2000)
    nonlinear_nk_result_['pyhrv'] = output['pyhrv nonlinear']['sd1/sd2']
    nonlinear_nk_result_['nk'] = output['nk nonlinear']['HRV_SD1SD2']
    nonlinear_nk_result[id] = nonlinear_nk_result_


nonlinear_biosppy_result = dict()
for row in tqdm(raw.iterrows(), total = len(raw)):
    id = row[0]
    sampling_rate = 2000
    signal = row[1].tolist()[start : end]
    pro = biosppy_processing(signal, sampling_rate)
    nonlinear_biosppy_result_ = dict()
    output = nonlinear(pro['rpeaks'], 2000)
    nonlinear_biosppy_result_['pyhrv'] = output['pyhrv nonlinear']['sd1/sd2']
    nonlinear_biosppy_result_['nk'] = output['nk nonlinear']['HRV_SD1SD2']
    nonlinear_biosppy_result[id] = nonlinear_biosppy_result_


nonlinear_aggregate_result = dict()
for row in tqdm(raw.iterrows(), total = len(raw)):
    id = row[0]
    sampling_rate = 2000
    signal = row[1].tolist()[start : end]
    pro = processing_aggregate(signal, sampling_rate)
    nonlinear_aggregate_result_ = dict()
    output = nonlinear(pro['rpeaks'], 2000)
    nonlinear_aggregate_result_['pyhrv'] = output['pyhrv nonlinear']['sd1/sd2']
    nonlinear_aggregate_result_['nk'] = output['nk nonlinear']['HRV_SD1SD2']
    nonlinear_aggregate_result[id] = nonlinear_aggregate_result_




0 ~ 120000


  0%|          | 0/70 [00:00<?, ?it/s]

  0%|          | 0/70 [00:00<?, ?it/s]

  0%|          | 0/70 [00:00<?, ?it/s]

In [14]:
print(f'{start} ~ {end}')
temp = nonlinear_nk_result
temp = pd.DataFrame(temp).T.describe().loc[['mean', 'std']].T
temp



0 ~ 120000


,mean,std
pyhrv,0.44,0.21
nk,0.44,0.21


In [15]:
print(f'{start} ~ {end}')
temp = nonlinear_biosppy_result
temp = pd.DataFrame(temp).T.describe().loc[['mean', 'std']].T
temp



0 ~ 120000


,mean,std
pyhrv,0.44,0.22
nk,0.44,0.22


In [16]:
print(f'{start} ~ {end}')
temp = nonlinear_aggregate_result
temp = pd.DataFrame(temp).T.describe().loc[['mean', 'std']].T
temp



0 ~ 120000


,mean,std
pyhrv,0.44,0.21
nk,0.44,0.21
